# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.120721e+02     1.458905e+00
 * time: 0.08944106101989746
     1     1.113597e+01     8.440378e-01
 * time: 0.9478020668029785
     2    -1.171424e+01     9.467217e-01
 * time: 1.076390027999878
     3    -3.408931e+01     7.845139e-01
 * time: 1.237112045288086
     4    -4.764510e+01     5.585544e-01
 * time: 1.388895034790039
     5    -5.690209e+01     1.850537e-01
 * time: 1.5347061157226562
     6    -5.978143e+01     1.179555e-01
 * time: 1.6414330005645752
     7    -6.085370e+01     5.183750e-02
 * time: 1.7499699592590332
     8    -6.125490e+01     6.596644e-02
 * time: 1.864617109298706
     9    -6.156589e+01     3.633605e-02
 * time: 1.971325159072876
    10    -6.178101e+01     2.740538e-02
 * time: 2.085245132446289
    11    -6.195361e+01     2.220664e-02
 * time: 2.1914050579071045
    12    -6.202664e+01     1.902294e-02
 * time: 2.305562973022461
    13    -6.210746e+01     1.810845e-02
 * time: 2.4116909503936768

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671055
    AtomicLocal         -18.8557627
    AtomicNonlocal      14.8522621
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485350 
    Xc                  -19.3336811

    total               -62.261666457556
